In [1]:
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = (
    SparkConf()
    .setAppName("knmi daily weather to bronze")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", 'minio')
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123')
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
#sqlContext = SQLContext(sc)
spark = SparkSession(sc).builder.getOrCreate()

23/02/13 21:02:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/13 21:02:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/13 21:02:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/13 21:02:21 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/13 21:02:21 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
df = spark.read.csv('s3a://landing-rws/*', header=True, sep=';')

23/02/13 21:02:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
df.show()

23/02/13 21:02:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+--------------+-------------+-------------+------------+-----------+---------+--------+-------------+----------------+--------+--------+--------+-----------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------------+-----------+--------------------+-------------------+---------+----------------+
|NLSitNummer|DatumFileBegin|DatumFileEind|TijdFileBegin|TijdFileEind|FileZwaarte|GemLengte|FileDuur|HectometerKop|HectometerStaart|RouteLet|RouteNum|RouteOms|hectometreringsrichting|        KopWegvakVan|       KopWegvakNaar|            TrajVan|            TrajNaar|  OorzaakGronddetail|      OorzaakVerloop|OorzaakCodeVerloop|OorzaakCode|           Oorzaak_1|          Oorzaak_2|Oorzaak_3|       Oorzaak_4|
+-----------+--------------+-------------+-------------+------------+-----------+---------+--------+-------------+----------------+--------+--------+--------+-----------------------+----

In [4]:
df.dropDuplicates().coalesce(1).write.format("delta").mode("overwrite").option("mergeSchema", "true").save('s3a://bronze-rws/filedata')
#df.write.format("csv").mode("overwrite").save('s3a://test-bucket/filedata')

In [5]:
df.dropDuplicates().coalesce(1).write.format("parquet").mode("overwrite").save('s3a://fontys/rws/filedata')